# Work in progress

In [56]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
from nibabel import trackvis
from dipy.tracking import utils

In [57]:
subj = 'F25_P2'
dname = 'E:/Celine/Ferret_b2n/'+subj+'/'
ftrk = dname + 'tracto_gtab_tricked_den_sd_2_peaks08.trk'
ffa = dname + 'tensor_fa_sd_2.nii.gz'

In [58]:
fa_img = nib.load(ffa)
fa = fa_img.get_data()
affine = fa_img.get_affine()

C:\Users\cdelettre\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: DeprecationWarning: get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  app.launch_new_instance()


In [117]:
streams, hdr = trackvis.read(ftrk,strict=True)
streamlines = [s[0] for s in streams]
streamlines = list(streamlines)

In [60]:
fmask = dname + 'mask_to_dwi.nii.gz' 
mask_img = nib.load(fmask)
mask = mask_img.get_data()

In [76]:
type(streams)

list

In [112]:
streamlines[0]

array([ 24.16425896,   6.89999962,   1.62      ], dtype=float32)

In [61]:
mask.shape

(128L, 60L, 40L)

In [62]:
# labels such as each voxel included in the mask is a roi

labels = mask.copy()

labels.shape
i=0

for z in range(mask.shape[2]):
    for y in range(mask.shape[1]):
        for index, item in enumerate(labels[:,y,z]):
            if item>0.0:
                i+=1
                labels[index,y,z]=i
labels = labels.astype(np.uint8)        

In [119]:
affine

array([[ 0.        ,  0.23999999,  0.        ,  0.        ],
       [-0.23984376,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.23999999,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [118]:
# can't use until we have labels

M, grouping = utils.connectivity_matrix(streamlines, labels, affine=affine,
                                        return_mapping=True,
                                        mapping_as_streamlines=True)

IndexError: streamline has points that map to negative voxel indices

In [ ]:
plt.imshow(np.log1p(M), interpolation='nearest')
plt.savefig("connectivity.png")